In [2]:
# libraries
import pandas as pd
import numpy as np
import geopandas as gpd

# classification dictionary
import dict_module

# set path
import os
os.chdir("poi_medicine")
print(f'current directory: {os.getcwd()}')

current directory: /Users/tomokitakata/Desktop/research/dev/muldistribution/poi_medicine


ベースの分類は東大病院の診療科構成を参考。  
- 腎臓・人工透析内科：これはこれで一つあるっぽい
- ペインクリニック：内科、麻酔科、整形外科に分類される場合がある。
- 平成 20 年 4 月 1 日以降、広告することが認められない診療科名「神経科」、「呼吸器科」、「消化器科」、「胃腸科」、「循環器科」、「皮膚泌尿器科」、「性病科」、「こう門科」、「気管食道科」

In [3]:
def match_word(keyword: str, pattern: str, match_type: str) -> bool:
    """
    部分一致/完全一致の判定関数
    """
    if match_type == "partial":
        return pattern in keyword
    elif match_type == "exact":
        return pattern == keyword
    return False

def check_rule(keyword: str, rule: dict) -> bool:
    """
    rule例:
      {
        "must": [("内科","partial"), ...],
        "any":  [("消化","partial"), ...],
        "not":  [("外科","partial"), ...]
      }
    - "must": 全部マッチする必要がある
    - "any":  いずれか1つマッチしていればOK（空ならスキップ）
    - "not":  1つでもマッチしたらNG
    """
    # must：全部マッチ必須
    for (pat, mtype) in rule.get("must", []):
        if not match_word(keyword, pat, mtype):
            return False

    # any：いずれかマッチ
    any_list = rule.get("any", [])
    if any_list:
        if not any(match_word(keyword, pat, mtype) for (pat, mtype) in any_list):
            return False

    # not：1つでもマッチしたらNG
    for (pat, mtype) in rule.get("not", []):
        if match_word(keyword, pat, mtype):
            return False

    return True


def classify_keywords(
    keywords, 
    classification_dict, 
    multiple=False,
    override_map=None,
    override_match_type="exact",
):
    """
    - keywords: 分類対象となる文字列のリスト
    - classification_dict: { "カテゴリ名": [rule1, rule2, ...], "分類不可": [] }
    - multiple: True の場合、複数カテゴリにマッチしても全て分類する
    - override_map: {"特定文字列": "分類先カテゴリ"} の辞書
    - override_match_type: "exact" or "partial"
        → override_map で「部分一致」させたいのか「完全一致」させたいのか指定
    """
    # 分類結果入れ物
    classified = {cat: [] for cat in classification_dict.keys()}

    for keyword in keywords:
        if not isinstance(keyword, str):
            # 文字列以外は強制的に「分類不可」
            classified["分類不可"].append(keyword)
            continue

        matched_categories = []

        # --------------------------
        # 1) まず override_map のチェック
        # --------------------------
        # 例: override_map = {"糖尿病内科": "糖尿病内科"}
        #    → キーワードが "糖尿病内科" なら強制的に "糖尿病内科" カテゴリへ
        override_assigned = False
        if override_map:
            for ov_key, ov_category in override_map.items():
                if match_word(keyword, ov_key, override_match_type):
                    classified[ov_category].append(keyword)
                    matched_categories.append(ov_category)
                    override_assigned = True
                    if not multiple:
                        break  # singleモードなら即打ち切り

            # override でマッチした場合は、通常のルール判定に進まず次のkeywordへ
            if override_assigned and not multiple:
                continue
            # override_assigned=True でも multiple=True の場合、以下のルール判定も継続する

        # --------------------------
        # 2) override で該当なしの場合、通常ルールで分類
        # --------------------------
        if not override_assigned or multiple:
            for category, rules in classification_dict.items():
                if category == "分類不可":
                    continue
                # いずれかのruleを満たせばOK
                if any(check_rule(keyword, r) for r in rules):
                    classified[category].append(keyword)
                    matched_categories.append(category)
                    if not multiple:
                        # singleモード (最初に見つかったカテゴリ1つだけ)
                        break

        # --------------------------
        # 3) いずれにもマッチしなかったら「分類不可」
        # --------------------------
        if not matched_categories:
            classified["分類不可"].append(keyword)

    return classified

In [11]:
# target word(s) : class(es) assigned

override_map_2010 = {
    "気管食道科": "呼吸器内科",  # 
    "リ": "リウマチ科",  # 市立札幌病院
    "神経科": '脳神経外科',
    "循環器科": '循環器内科',
    '呼吸器科': '呼吸器内科',
}

In [ ]:
# 分類対象となるキーワードの例
uniq_dpt = pd.read_excel('A_Unique_departments_unclassi/2010.xlsx').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()

classified_keywords = classify_keywords(
    keywords=uniq_dpt,
    classification_dict=dict_module.classification_dict,
    multiple=True,            # 複数マッチを許可
    override_map=override_map_2010,
    override_match_type="exact"   # override判定は完全一致
)

# 分類結果をDataFrameに整形して表示
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
result_df

,総合内科,内科,呼吸器内科,消化器内科,循環器内科,腎臓・内分泌内科,糖尿病・代謝内科,血液・腫瘍内科,アレルギー科,リウマチ科,...,口腔顎顔面外科・歯科,小児科,小児外科,産婦人科,精神神経科,放射線科,救急・集中治療科,臨床腫瘍科,病理診断科,分類不可
0,内科,内科,呼吸器科,消化器科,循環器科,人工透析内科,,脳血管内科,アレルギー科,リウマチ科,...,歯科,小児科,小児外科,産婦人科,精神科,放射線科,,,,
1,神経内科,,気管食道科,胃腸科,循環器内科,,,消化器・血液内科,,リ,...,歯科口腔外科,小児外科,内科・小児科・整形外科・外科・歯科・リハビリテーション科,婦人科,,放射,,,,
2,心療内科,,呼吸器内科,消化器,,,,,,,...,歯科口,小児歯科,,産科,,放射線,,,,
3,脳血管内科,,,消,,,,,,,...,小児歯科,内科・小児科・整形外科・外科・歯科・リハビリテーション科,,,,,,,,
4,循環器内科,,,消化器・血液内科,,,,,,,...,矯正歯科,,,,,,,,,
5,呼吸器内科,,,,,,,,,,...,歯,,,,,,,,,
6,肝臓内科,,,,,,,,,,...,内科・小児科・整形外科・外科・歯科・リハビリテーション科,,,,,,,,,
7,人工透析内科,,,,,,,,,,...,,,,,,,,,,
8,消化器・血液内科,,,,,,,,,,...,,,,,,,,,,
9,内科・小児科・整形外科・外科・歯科・リハビリテーション科,,,,,,,,,,...,,,,,,,,,,


In [15]:
os.getcwd()

'/Users/tomokitakata/Desktop/research/dev/muldistribution/poi_medicine'

In [16]:
result_df.to_csv('../2010_classified.csv', encoding='utf-8', index=True)

診療科の分類は、医療レセプトにおける厚生労働省が指定する区分を使用。
https://www.mhlw.go.jp/topics/2009/05/dl/tp0521-1a_0053.pdf